# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
browser.implicitly_wait(1)

## 기사 링크 크롤링

In [127]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

six_url_list = []   # url 저장 리스트
entertainment_url_list = []
sports_url_list = []

category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

def getSixLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    a_list = []

    for category in range(3, 4):     # 6
        for page in range(1, 7):  # 1, 6
            url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
            browser.get(url)

            # time.sleep(1)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".type06_headline dt+dt a"))
            a_list.extend(soup.select(".type06 dt+dt a"))

            category_list.extend([categoty_name[category] for _ in range(20)])
            print(f"{categoty_name[category]} {page} 페이지")

    for a in a_list:
        six_url_list.append(a["href"])


def getEntertainmentLink():   # 연예
    # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
    a_list = []
    today = datetime.date.today()

    for page in range(1, 5):  # 1, 5
        url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
        browser.get(url)

        # time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        a_list.extend(soup.select(".news_lst li>a"))

        category_list.extend(["연예" for _ in range(25)])
        print(f"연예 {page} 페이지")

    for a in a_list:
        entertainment_url_list.append("https://entertain.naver.com" + a["href"])


def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
    # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
    a_list = []
    today = str(datetime.date.today()).replace('-', '')

    for page in range(1, 2):  # 1, 7
        url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
        browser.get(url)

        # time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".news_list li>a"))

        print(f"스포츠 {page} 페이지")

    for i in range(len(a_list)):
        if i == 100:  # 100개 링크 추가했으면 멈추기
            break
        sports_url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
        category_list.extend(["스포츠"])

getSixLink()
# getEntertainmentLink()
# getSportsLink()


생활/문화 1 페이지
생활/문화 2 페이지
생활/문화 3 페이지
생활/문화 4 페이지
생활/문화 5 페이지
생활/문화 6 페이지


In [128]:
print(len(six_url_list))
print(len(entertainment_url_list))
print(len(sports_url_list))

120
0
0


## 제목, 본문, 날짜 크롤링

In [105]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w{2,4}기자','',article)
    article = re.sub('사진','',article)
    article = re.sub('포토','',article)
    # article = re.sub('\S+@[a-z.]+','',article)

    article = re.sub('\n','',article)
    article = re.sub('\t','',article)
    article = re.sub('\u200b','',article)
    article = re.sub('\xa0','',article)
    article = re.sub('([a-zA-Z])','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)


    return article


In [129]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

def getSixContent(url_list):   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        # time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select("#title_area span"))                 # 제목

        c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

    for t in title:
        title_list.append(clean(t.text))

    for c in content:
        content_list.append(clean(c.text))

    for d in date:
        date_list.append(d.text)


def getEntertainmentContent(url_list):    # 연예
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        # time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".end_tit"))                 # 제목

        c = soup.find_all(attrs={"id" : "articeBody"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기
          if c[0].find(attrs={"class" : "caption"}):                  # 이미지 설명 없애기
            c[0].find(attrs={"class" : "caption"}).decompose()

        while c[0].find(attrs={"class" : "video_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "video_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        while c[0].find(attrs={"name" : "iframe"}):
           c[0].find(attrs={"name" : "iframe"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".author em"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)


def getSportsContent(url_list):   # 스포츠
    title = []
    content = []
    date = []

    for url in url_list:

        browser.get(url)
        print(url)
        # time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".news_headline .title"))                 # 제목

        c = soup.find_all(attrs={"class" : "news_end"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "image"}):
           c[0].find(attrs={"class" : "image"}).decompose()

        while c[0].find(attrs={"class" : "vod_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        c[0].find(attrs={"class" : "source"}).decompose()
        c[0].find(attrs={"class" : "byline"}).decompose()
        c[0].find(attrs={"class" : "reporter_area"}).decompose()
        c[0].find(attrs={"class" : "copyright"}).decompose()
        c[0].find(attrs={"class" : "categorize"}).decompose()
        c[0].find(attrs={"class" : "promotion"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".info span"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)

getSixContent(six_url_list)
# getEntertainmentContent(entertainment_url_list)
# getSportsContent(sports_url_list)


In [131]:
print(len(title_list))
print(len(content_list))
print(len(date_list))

120
120
120


## 데이터 프레임 생성

In [132]:
# 데이터 프레임으로 변환
all_url_list = six_url_list + entertainment_url_list + sports_url_list

article_df = pd.DataFrame({"category" : category_list,
                           "date" : date_list,
                           "title" : title_list,
                           "content" : content_list,
                           "url" : all_url_list})

article_df

,category,date,title,content,url
0,생활/문화,2023.11.03. 오후 8:01,美 유명 코미디언 밤마다 이 통조림’ 3캔 먹었더니 혈액서 중금속 검출,해외토픽미국 유명 코미디언이자 팟캐스터인 조 로건이 밤마다 정어리 통조림을 먹은 뒤...,https://n.news.naver.com/mnews/article/346/000...
1,생활/문화,2023.11.03. 오후 7:43,쌓인 낙엽으로 물웅덩이 생겨 불편초래인근 주민이 배수로 뚫어,천둥과 번개를 동반한 비가 쏟아지자 쌓여있던 낙엽이 배수로를 막아 길이 잠기면서 시...,https://n.news.naver.com/mnews/article/654/000...
2,생활/문화,2023.11.03. 오후 7:37,퇴근길 날씨 내일 강원 영동·남부지방 비모레 밤부터 강한 비바람,현재 서울과 강원에 시간당 10 안팎의 비가 내리고 있습니다 오늘 밤까지 중부지방...,https://n.news.naver.com/mnews/article/056/001...
3,생활/문화,2023.11.03. 오후 7:35,책마을 500년전 394명 처형한 사형집행인의 근무일지,뉘른베르크의 사형집행인조엘 해링톤 지음이지안 옮김 마르코폴로420쪽│3만원독일 뉘...,https://n.news.naver.com/mnews/article/015/000...
4,생활/문화,2023.11.03. 오후 7:13,4인가족 한끼 80만원특급호텔 뷔페 연말 맞아 가격 인상,연말을 맞아 서울 시내 주요 특급호텔들의 뷔페 가격이 처음으로 20만 원을 넘어섰다...,https://n.news.naver.com/mnews/article/008/000...
...,...,...,...,...,...
115,생활/문화,2023.11.03. 오전 11:19,56일 중부·제주 최대 100㎜ 많은 비이후 반짝추위,중국 내륙서 발달한 저기압 북한 쪽으로 지나며 전국에 비시간당 2040㎜ 집중호우 ...,https://n.news.naver.com/mnews/article/001/001...
116,생활/문화,2023.11.03. 오전 11:15,“최대 700만원대 할인”한국지엠 11월 쉐보레 빅 페스타’ 프로모션,한국지엠이 11월 쉐보레 신차에 대해 700만원에 달하는 할인 혜택을 제공한다고 3...,https://n.news.naver.com/mnews/article/009/000...
117,생활/문화,2023.11.03. 오전 11:15,스포츠브라가 유방암 유발한다 주장에 조회수 폭발진실은 헬스톡,파이낸셜뉴스 운동을 할 때 스포츠 브라를 입으면 가슴에 혹이 생길 수 있다고 주장...,https://n.news.naver.com/mnews/article/014/000...
118,생활/문화,2023.11.03. 오전 11:12,피곤하면 입 안에 생기는 물집 종류 여러 개라고,평소보다 무리하거나 스트레스를 받으면 입에 물집이 생기는 경우가 많다 보통 잇몸 혀...,https://n.news.naver.com/mnews/article/346/000...


# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class


In [ ]:
pip install konlpy

In [59]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt


# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [133]:
def getNouns():
    okt = Okt()
    nouns_list = []   # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

    article_df["nouns"] = nouns_list   # 데이터 프레임에 추가


getNouns()

In [134]:
def getVector():
    text = [" ".join(noun) for noun in article_df["nouns"]]  # 명사 열을 하나의 리스트에 담는다.

    tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 5))
    tfidf_vectorizer.fit(text)
    vector = tfidf_vectorizer.transform(text).toarray()  # vector list 반환
    vector = np.array(vector)

    return vector

vector = getVector()

# 군집화

**DBSCAN**
- https://bcho.tistory.com/1205


In [42]:
from sklearn.cluster import DBSCAN

# DBSCAN : 밀도 기반 클러스터링
# 점 p에서 부터 거리 e (epsilon)내에 점이 m(minPts) 개 있으면 하나의 군집으로 인식

In [135]:
model = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
result = model.fit_predict(vector)
result

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
       14, 15, 16, 17, 18, 19, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
       30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53, 54, 55, 20, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
       80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 67, 93, 94, 95,
       96])

In [136]:
test_df = pd.DataFrame({"ClusterNumber" : result,
                        "title": title_list,
                        "url": all_url_list
                        })

In [137]:
test_df

,ClusterNumber,title,url
0,0,美 유명 코미디언 밤마다 이 통조림’ 3캔 먹었더니 혈액서 중금속 검출,https://n.news.naver.com/mnews/article/346/000...
1,1,쌓인 낙엽으로 물웅덩이 생겨 불편초래인근 주민이 배수로 뚫어,https://n.news.naver.com/mnews/article/654/000...
2,2,퇴근길 날씨 내일 강원 영동·남부지방 비모레 밤부터 강한 비바람,https://n.news.naver.com/mnews/article/056/001...
3,3,책마을 500년전 394명 처형한 사형집행인의 근무일지,https://n.news.naver.com/mnews/article/015/000...
4,4,4인가족 한끼 80만원특급호텔 뷔페 연말 맞아 가격 인상,https://n.news.naver.com/mnews/article/008/000...
...,...,...,...
115,67,56일 중부·제주 최대 100㎜ 많은 비이후 반짝추위,https://n.news.naver.com/mnews/article/001/001...
116,93,“최대 700만원대 할인”한국지엠 11월 쉐보레 빅 페스타’ 프로모션,https://n.news.naver.com/mnews/article/009/000...
117,94,스포츠브라가 유방암 유발한다 주장에 조회수 폭발진실은 헬스톡,https://n.news.naver.com/mnews/article/014/000...
118,95,피곤하면 입 안에 생기는 물집 종류 여러 개라고,https://n.news.naver.com/mnews/article/346/000...


In [138]:
for n in set(result):   # 군집 결과
    print(f"ClusterNumber : {n}")   # 군집 번호
    temp_df = test_df[test_df["ClusterNumber"] == n]
    for title in temp_df["title"]:
        print(title)
    print()

ClusterNumber : 0
美 유명 코미디언 밤마다 이 통조림’ 3캔 먹었더니 혈액서 중금속 검출
美 유명 코미디언 밤마다 이 통조림’ 3캔 먹었더니 혈액서 중금속 검출

ClusterNumber : 1
쌓인 낙엽으로 물웅덩이 생겨 불편초래인근 주민이 배수로 뚫어
쌓인 낙엽으로 물웅덩이 생겨 불편초래인근 주민이 배수로 뚫어

ClusterNumber : 2
퇴근길 날씨 내일 강원 영동·남부지방 비모레 밤부터 강한 비바람
퇴근길 날씨 내일 강원 영동·남부지방 비모레 밤부터 강한 비바람

ClusterNumber : 3
책마을 500년전 394명 처형한 사형집행인의 근무일지
책마을 500년전 394명 처형한 사형집행인의 근무일지

ClusterNumber : 4
4인가족 한끼 80만원특급호텔 뷔페 연말 맞아 가격 인상
4인가족 한끼 80만원특급호텔 뷔페 연말 맞아 가격 인상

ClusterNumber : 5
책마을 미드웨이 해전 졌더라도 美 승리는 시간 문제였다
책마을 미드웨이 해전 졌더라도 美 승리는 시간 문제였다

ClusterNumber : 6
자주 먹는 이 음식’ 삶을 불행하게 만든다
자주 먹는 이 음식’ 삶을 불행하게 만든다

ClusterNumber : 7
방광암 새 치료법 나왔다 사망 위험 절반으로
방광암 새 치료법 나왔다 사망 위험 절반으로

ClusterNumber : 8
생활날씨 주말 충청 이남 비모레 중부 제주 호우
생활날씨 주말 충청 이남 비모레 중부 제주 호우

ClusterNumber : 9
효과 좋대서 먹은 기침 가래약 나만 효과 없는 이유 있었다이게뭐약
효과 좋대서 먹은 기침 가래약 나만 효과 없는 이유 있었다이게뭐약

ClusterNumber : 10
좋은데 왜 안 하지의사 왕진 직접 신청해보니
좋은데 왜 안 하지의사 왕진 직접 신청해보니

ClusterNumber : 11
날씨 늦더위 이어 가을 호우다음 주에는 초겨울 추위
날씨 늦더위 이어 가을 호우다음 주에는 초겨울 추위

ClusterNumber : 12
중국 가 주목한 